In [2]:
# Install necessary libraries quietly (-q suppresses output)
!pip install gradio pillow easyocr -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.8/57.8 MB 13.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 321.9/321.9 kB 26.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 92.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 101.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.5/71.5 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 61.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 33.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 45.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3

In [3]:
# Import necessary libraries
import os # Provides access to environment variables and file system operations
import numpy as np  # NumPy for numerical operations and array manipulation
from PIL import Image  # Image processing library
import easyocr  # Optical Character Recognition (OCR) for text extraction from images
import requests  # For making HTTP requests (e.g., fetching data from URLs)
import json  # Handling JSON data
import gradio as gr  # Building interactive web-based applications

In [4]:
# Initialize OCR reader
reader = easyocr.Reader(['en'])

# DeepSeek API configuration
from google.colab import userdata

API_KEY = userdata.get('deepseek_api')  # Retrieves the secret from environment variables
DEEPSEEK_API_URL = "https://api.deepseek.com/v1/chat/completions"

def extract_numbers(image_path):
    """Extracts all characters from an image and arranges them into a 3x3 grid, replacing non-digits with '?'."""
    results = reader.readtext(image_path, detail=1)  # Get bounding boxes with text

    detected_cells = []
    for (bbox, text, prob) in results:
        # Replace non-digit characters with '?'
        detected_cells.append(text if text.isdigit() else '?')  # Collect all detected text, replace non-digits with '?'

    if not detected_cells:
        return [['?', '?', '?'], ['?', '?', '?'], ['?', '?', '?']]  # Return a grid with '?' if nothing is detected

    # Step 1: Create a 3x3 grid and fill it with the detected characters row by row
    grid = [['?', '?', '?'], ['?', '?', '?'], ['?', '?', '?']]  # Initialize with '?' as placeholders
    i = 0  # Start filling from the first detected character

    # Step 2: Place the detected characters in the grid
    for row in range(3):
        for col in range(3):
            if i < len(detected_cells):
                grid[row][col] = detected_cells[i]  # Place the detected character in the grid
                i += 1

    return grid

def format_puzzle_for_deepseek(grid):
    """Formats the extracted puzzle with variable placeholders for missing values."""
    variables = {}
    variable_index = 0
    formatted_grid = []

    for row in grid:
        formatted_row = []
        for cell in row:
            if cell == "?":
                variable_name = chr(65 + variable_index)  # Assign 'A', 'B', 'C'...
                variables[variable_name] = "?"
                formatted_row.append(variable_name)
                variable_index += 1
            else:
                formatted_row.append(cell)
        formatted_grid.append(formatted_row)

    return formatted_grid, variables

def solve_with_deepseek(formatted_grid, variables):
    """Sends the formatted puzzle to DeepSeek-Chat for step-by-step solving."""
    puzzle_text = "\n".join(["\t".join(row) for row in formatted_grid])
    prompt = (
        "You are an AI specialized in solving magic square puzzles. Analyze the following grid, where missing values are already "
        "assigned as variables (e.g., A, B, C, D, E). Provide the correct solution with step-by-step reasoning. Ensure the sum of "
        "each row, column, and diagonal is the same. Duplicates are not allowed in any row, column, or diagonal.\n\n"
        f"Given Question:\n{puzzle_text}\n\n"
        "The missing values are represented as variables (e.g., A, B, C, D, E). Provide a structured solution."
        "\nFormat your response strictly as follows:\n"
        "1. **Given Question**:\n   - (grid format with missing values as variables)\n"
        "2. **Step-by-step Reasoning**:\n   - Identify the missing values logically\n   - Explain the rules used\n   - Calculate step-by-step\n"
        "3. **Final Answer**:\n   - Completed grid\n"
        "4. **Validation**:\n   - Verify all row/column sums\n"
    )

    data = {
        "model": "deepseek-chat",
        "messages": [{"role": "user", "content": prompt}],
        "temperature": 0,
        "max_tokens": 2000
    }

    headers = {
        "Authorization": f"Bearer {API_KEY}",
        "Content-Type": "application/json"
    }

    try:
        response = requests.post(DEEPSEEK_API_URL, headers=headers, json=data, timeout=300)
        response.raise_for_status()
        result = response.json()
        return result.get("choices", [{}])[0].get("message", {}).get("content", "Error: No response content.")
    except requests.exceptions.Timeout:
        return "Error: DeepSeek API request timed out. Please try again."
    except requests.exceptions.RequestException as e:
        return f"Error: DeepSeek API failed: {str(e)}"


def solve_magic_square(image):
    """Main function to extract, format, solve, and validate a magic square puzzle."""
    image_path = "uploaded_magic_square.png"
    image.save(image_path)
    extracted_grid = extract_numbers(image_path)
    formatted_grid, variables = format_puzzle_for_deepseek(extracted_grid)
    solution = solve_with_deepseek(formatted_grid, variables)
    return solution

Progress: |██████████████████████████████████████████████████| 100.0% Complete

Progress: |██████████████████████████████████████████████████| 100.0% Complete

In [ ]:
# Create a Gradio interface
iface = gr.Interface(
    fn=solve_magic_square,  # Function to call for OCR and solving
    inputs=gr.Image(type="pil"),  # Input type: PIL image
    outputs=gr.Textbox(),  # Output type: Textbox to display extracted and solved result
    title="Magic Square Puzzle Solver",  # Title of the interface
    description=(
        "Upload an image of a 3x3 Magic Square puzzle. \n\n"
        "- A Magic Square is a 3x3 grid where the sum of each row, column, and diagonal is the same. \n"
        "- The puzzle involves solving for missing values represented by variables (e.g., A, B, C, D, E). \n"
        "- Please note that this tool only works with 3x3 puzzles. Duplicates are not allowed in any row, column, or diagonal.\n"
    ),  # Detailed description of the game rules
    allow_flagging="never"  # Disables the flagging feature for now
)

# Launch the Gradio interface
iface.launch(debug=True)

/usr/local/lib/python3.11/dist-packages/gradio/interface.py:403: UserWarning: The `allow_flagging` parameter in `Interface` is deprecated.Use `flagging_mode` instead.
  warnings.warn(


Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://16d4506540d0cfd9a0.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
